# 🎬 IMDb Movie Chatbot - Google Colab Version

This notebook implements an AI-powered movie recommendation chatbot using:
- **RAG (Retrieval-Augmented Generation)** for accurate movie information
- **FAISS Vector Search** for fast similarity matching
- **OpenAI GPT** for natural language understanding
- **Agentic AI Architecture** with specialized movie tools

---

## Setup Instructions
1. Run Cell 1 to install dependencies
2. Run Cell 2 to mount Google Drive (upload your dataset there)
3. Run Cell 3 to set your OpenAI API key
4. Run all remaining cells in order
5. Launch the Gradio UI in the final cell

---

In [ ]:
# Cell 1: Install Dependencies (Run this first!)
# This will take 1-2 minutes

!pip install -q langchain langchain-openai langchain-community faiss-cpu gradio python-dotenv openai

print("✅ All dependencies installed successfully!")

In [ ]:
# Cell 2: Mount Google Drive & Set Dataset Path
# Upload your IMDb_Dataset.csv to Google Drive first

from google.colab import drive
drive.mount('/content/drive')

# UPDATE THIS PATH to match your Google Drive location
# Example: '/content/drive/MyDrive/IMDb_Dataset.csv'
DATASET_PATH = '/content/drive/MyDrive/IMDb_Dataset (1).csv'

# Alternative: Upload directly to Colab (uncomment below)
# from google.colab import files
# uploaded = files.upload()  # This will prompt you to upload
# DATASET_PATH = list(uploaded.keys())[0]

print(f"Dataset path set to: {DATASET_PATH}")

In [ ]:
# Cell 3: Set OpenAI API Key
# Option A: Use Colab Secrets (Recommended)
# Option B: Enter directly (less secure)

import os

# Option A: Try to get from Colab Secrets first
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except:
    OPENAI_API_KEY = None

# Option B: If not in secrets, prompt for input
if not OPENAI_API_KEY:
    from getpass import getpass
    OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
    print("✅ API key entered manually")

# Set the environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ OpenAI API key configured!")

In [ ]:
# Cell 4: Import Libraries

import os
import pandas as pd
import numpy as np
from typing import Tuple, List, Dict, Optional

# LangChain imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.memory import ConversationBufferMemory

# Gradio for UI
import gradio as gr

# Warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

In [ ]:
# Cell 5: Load the Dataset

# Load the IMDb dataset
df = pd.read_csv(DATASET_PATH)

print(f"✅ Dataset loaded successfully!")
print(f"Shape: {df.shape[0]} movies, {df.shape[1]} features")
print(f"\nColumns: {list(df.columns)}")

# Show sample
df.head()

In [ ]:
# Cell 6: Exploratory Data Analysis

print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)

# Data types and non-null counts
print("\n📋 Data Types & Missing Values:")
print(df.info())

# Statistical summary
print("\n📈 Statistical Summary:")
display(df.describe())

# Missing values
print("\n⚠️ Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Percentage': missing_pct})
display(missing_df[missing_df['Missing Count'] > 0])

# Key stats
print(f"\n🎭 Unique Values:")
print(f"- Genres: {df['Genre'].nunique()} unique")
print(f"- Directors: {df['Director'].nunique()} unique")
print(f"- Year range: {df['Year'].min()} - {df['Year'].max()}")
print(f"- Rating range: {df['IMDb Rating'].min()} - {df['IMDb Rating'].max()}")

In [ ]:
# Cell 7: Create Movie Descriptions

def create_movie_description(row):
    """Create a rich text description for each movie."""
    title = row['Title'] if pd.notna(row['Title']) else 'Unknown Title'
    year = int(row['Year']) if pd.notna(row['Year']) else 'Unknown Year'
    genre = row['Genre'] if pd.notna(row['Genre']) else 'Unknown Genre'
    director = row['Director'] if pd.notna(row['Director']) else 'Unknown Director'
    cast = row['Star Cast'] if pd.notna(row['Star Cast']) else 'Unknown Cast'
    rating = row['IMDb Rating'] if pd.notna(row['IMDb Rating']) else 'N/A'
    metascore = row['MetaScore'] if pd.notna(row['MetaScore']) else 'N/A'
    certificate = row['Certificates'] if pd.notna(row['Certificates']) else 'Not Rated'
    duration = int(row['Duration (minutes)']) if pd.notna(row['Duration (minutes)']) else 'Unknown'
    poster = row['Poster-src'] if pd.notna(row['Poster-src']) else ''
    
    description = f"""
Movie Title: {title}
Year: {year}
Genre: {genre}
Director: {director}
Star Cast: {cast}
IMDb Rating: {rating}/10
MetaScore: {metascore}
Certificate: {certificate}
Duration: {duration} minutes

This is a {genre} movie titled "{title}" released in {year}. 
It was directed by {director} and stars {cast}. 
The film has an IMDb rating of {rating}/10 and a MetaScore of {metascore}.
""".strip()
    
    return description

print("Creating movie descriptions...")
df['description'] = df.apply(create_movie_description, axis=1)

print(f"\n✅ Created descriptions for {len(df)} movies")
print(f"\nSample description:")
print("-" * 50)
print(df['description'].iloc[0])

In [ ]:
# Cell 8: Create Documents for Vector Store

def create_documents_from_dataframe(df):
    """Convert DataFrame rows to LangChain Document objects."""
    documents = []
    
    for idx, row in df.iterrows():
        metadata = {
            'title': row['Title'] if pd.notna(row['Title']) else 'Unknown',
            'year': int(row['Year']) if pd.notna(row['Year']) else 0,
            'genre': row['Genre'] if pd.notna(row['Genre']) else 'Unknown',
            'director': row['Director'] if pd.notna(row['Director']) else 'Unknown',
            'rating': float(row['IMDb Rating']) if pd.notna(row['IMDb Rating']) else 0.0,
            'certificate': row['Certificates'] if pd.notna(row['Certificates']) else 'Not Rated',
            'poster_url': row['Poster-src'] if pd.notna(row['Poster-src']) else '',
            'index': idx
        }
        
        doc = Document(page_content=row['description'], metadata=metadata)
        documents.append(doc)
    
    return documents

print("Converting to documents...")
documents = create_documents_from_dataframe(df)
print(f"\n✅ Created {len(documents)} documents")

In [ ]:
# Cell 9: Create Embeddings Model

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("✅ OpenAI Embeddings initialized")
print("   Model: text-embedding-3-small")

# Quick test
test_embed = embeddings.embed_query("action movie")
print(f"   Embedding dimensions: {len(test_embed)}")

In [ ]:
# Cell 10: Create FAISS Vector Store
# This cell may take 2-5 minutes for ~3000 movies

print("Creating FAISS vector store...")
print(f"Processing {len(documents)} documents (this may take a few minutes)...")

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)

print(f"\n✅ Vector store created with {vectorstore.index.ntotal} vectors")

# Test search
print("\n🔍 Testing search...")
test_results = vectorstore.similarity_search("comedy movie", k=3)
for i, doc in enumerate(test_results, 1):
    print(f"   {i}. {doc.metadata['title']} ({doc.metadata['year']})")

In [ ]:
# Cell 11: Create LLM and RAG Chain

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=1000,
)
print("✅ LLM initialized (gpt-4o-mini)")

# Create prompt template
MOVIE_PROMPT = """You are an expert Movie Recommendation Assistant with access to the IMDb database.

Use the following movie information to answer the user's question:

{context}

Guidelines:
1. Only recommend movies from the provided context
2. Provide details like title, year, genre, director, cast, and ratings
3. Be conversational and helpful
4. If no relevant movies found, say so politely

User Question: {question}

Answer:"""

prompt = PromptTemplate(template=MOVIE_PROMPT, input_variables=["context", "question"])

# Create chains
combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
retrieval_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=combine_docs_chain)

print("✅ RAG chain assembled")

In [ ]:
# Cell 12: Test the RAG Chain

def ask_movie_bot(question: str) -> dict:
    """Process a question through the RAG chain."""
    return retrieval_chain.invoke({"input": question})

# Test
print("🎬 Testing Movie Chatbot")
print("=" * 50)

test_q = "Recommend some good documentary movies"
print(f"\n📝 Query: {test_q}")
response = ask_movie_bot(test_q)
print(f"\n🤖 Response:\n{response['answer']}")

In [ ]:
# Cell 13: Create Agent Tools

@tool
def search_movies_by_query(query: str) -> str:
    """Search for movies based on a natural language query."""
    response = retrieval_chain.invoke({"input": query})
    return response['answer']

@tool
def get_movie_details(movie_title: str) -> str:
    """Get detailed information about a specific movie by title."""
    query = f"Tell me everything about the movie titled {movie_title}"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']

@tool
def recommend_movies_by_genre(genre: str) -> str:
    """Recommend movies from a specific genre."""
    query = f"Recommend the best {genre} movies with high ratings"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']

@tool
def find_movies_by_actor(actor_name: str) -> str:
    """Find movies featuring a specific actor."""
    query = f"Find movies starring {actor_name}"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']

@tool
def find_movies_by_director(director_name: str) -> str:
    """Find movies by a specific director."""
    query = f"Find movies directed by {director_name}"
    response = retrieval_chain.invoke({"input": query})
    return response['answer']

tools = [
    search_movies_by_query,
    get_movie_details,
    recommend_movies_by_genre,
    find_movies_by_actor,
    find_movies_by_director,
]

print(f"✅ Created {len(tools)} agent tools")

In [ ]:
# Cell 14: Create Agent Orchestrator

AGENT_PROMPT = """You are a helpful Movie Recommendation Assistant powered by the IMDb database.
You have tools to search movies, get details, recommend by genre, and find by actor/director.
Always be helpful and provide detailed movie information."""

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", AGENT_PROMPT),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=agent_prompt)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=False,
    handle_parsing_errors=True,
    max_iterations=5,
)

print("✅ Agent Orchestrator created")

In [ ]:
# Cell 15: Create Chatbot Class with Error Handling

class MovieChatbot:
    def __init__(self, agent_executor):
        self.agent = agent_executor
        self.history = []
        
    def chat(self, user_input: str) -> str:
        if not user_input or len(user_input.strip()) < 3:
            return "⚠️ Please enter a more detailed question about movies."
        
        if len(user_input) > 1000:
            return "⚠️ Your question is too long. Please keep it under 1000 characters."
        
        try:
            response = self.agent.invoke({"input": user_input})
            self.history.append({"user": user_input, "assistant": response['output']})
            return response['output']
        except Exception as e:
            return f"⚠️ An error occurred: {str(e)}"
    
    def clear_history(self):
        self.history = []
        self.agent.memory.clear()
        return "History cleared!"

chatbot = MovieChatbot(agent_executor)
print("✅ Chatbot ready!")

# Test
print("\n🧪 Test: ")
print(chatbot.chat("Recommend a documentary"))

In [ ]:
# Cell 16: Launch Gradio Interface
# Run this cell to start the chatbot UI!

def chat_fn(message: str, history: list) -> str:
    return chatbot.chat(message)

demo = gr.ChatInterface(
    fn=chat_fn,
    title="🎬 IMDb Movie Chatbot",
    description="Ask me anything about movies! I can recommend films, find movies by genre, director, or actors.",
    examples=[
        "Recommend some highly rated documentaries",
        "Find movies starring Tom Hanks",
        "What are some good adventure movies?",
        "Tell me about biography films",
    ],
    theme="soft"
)

# Launch with public URL for sharing
demo.launch(share=True)

---

## 🎉 Congratulations!

Your IMDb Movie Chatbot is now running!

### Features:
- **RAG-powered** responses using your IMDb dataset
- **Agentic AI** with specialized movie tools
- **Conversation memory** for context-aware responses
- **Error handling** for robust operation

### Try asking:
- "Recommend a good documentary"
- "Find movies by Christopher Nolan"
- "What are the highest rated films?"
- "Movies starring Robert De Niro"

---

*Built with LangChain, FAISS, OpenAI, and Gradio*